# Prioritized Experience Replay
Prioritized Experience Replay (PER) is an improvement on DQN [1] that was introduced by [2]. The authors of [2] suggest that it improves the agent by sampling experience according to its significance instead of uniformly random. The measure that is chosen for priority is the TD-error; experience samples that are less well described by the qnet are sampled more often.

One part of the algorithm that is necessary for performant sampling with priority is the Segment Tree datastructure. While [2] does not go into much detail on the implementation of the Segment Tree, I've found various resources that really helped me understand how it works: [3], [4] and [5].

## Results

I first implemented prioritized experience replay with a fixed value of beta.
The parameters I first tried were roughly inspired by the PER manuscript: I chose a smaller learning rate of 5e-4, alpha = 0.6.  beta started with a value of 0.4 and increased to 1 within 60,000 steps. For the buffer size I started with 2e17, which is roughly 2/3 of the buffer size in the previous experiments.

After being unsatisfied with the results, I started experimenting with some of the parameters. Since the calculation times with PER are much longer now I focused on only one of the experiments for tuning the hyperparameters, cartpole_scale which didn't have great results before.

One thing that seemed to help was increasing the learning rate back to 1e-3 while incrementing beta much more slowly with an increment of 1e-6 per step.  
The first parameter set I was happy with had a buffer size of 2e18, which I thought was a bit excessive, since in the best case, the agent has 10e6 steps in the cartpole swingup environment.
I decreased the buffer size to 2e16 and hoped that the experiment would finish faster.
The compute time was only slightly better with a smaller replay buffer (5:03h vs. 5:21h), but the performance was better with this smaller replay buffer. T

Thus these are the results I will compare with the Dueling DQN agent. The settings I finally used are:  
<em> Run024  
qnet_settings = {"layers_sizes": [50], "batch_size": 64}

settings = {"batch_size": qnet_settings["batch_size"], "epsilon_start": 1.0, "epsilon_decay": 0.999,
            "epsilon_min": 0.025, "gamma": 0.99, "buffer_size": 2\**16, "lr": 1e-3, "qnet_settings": qnet_settings,
            "start_optimization": 64, "update_qnet_every": 2, "update_target_every": 50,
            "ddqn": True, "n_steps": 4, "duelling_dqn": True, "prioritized_buffer" : True, "alpha": 0.5,
            "beta0": 0.5, "beta_increment": 1e-6}
</em>

![Radar plot without (run019) and with prioritized experience replay (run024)](./figures/per_radar.png)

![Bar plot comparing the different experiments without (run019) and with prioritized experience replay (run024).](./figures/per_barplot.png)

The results are much better for the cartpole_swingup experiment, which is kind of obvious, since I used that experiment to tune my hyperparameters. In particular, as you can see below, several of the Cartpole Swingup runs managed to get a positive return, while the agent without prioritized experience replay did only have one successful episode.
The PER agent also shows better results on some of the cartpole and mountain car experiments but is otherwise mostly comparable to the DuelDQN agent.

![Episodic return for the cartpole swingup experiment without (run019) and with prioritized experience replay (run024).](./figures/per_cartpole_swingup_return.png)



## Discussion
Typically, using a larger replay buffer is advantageous. Maybe using a smaller replay buffer is better in this instance since the replay buffer is still large enough to keep samples for a significant time.
Eventually discarding old samples with a large TD-error and using more recent samples in the optimization step instead could be helpful.

Improving the replay buffer sampling is a broad field. One could add further criteria besides TD-error to improve the quality;
- increase the probability with the time a sample is not used 
- decrease the probability proportional to the number of times a sample has been used in the optimization step

Otherwise these experiments again hinted at parameter tuning being very important. My plan is to finish implementing the remaining improvements (Noisy Nets and Distributional DQN), tune the hyperparameters for the final agent as well as the base agent and then compare these two agents.

## References
[1] Mnih, Volodymyr, et al. Human-level control through deep reinforcement learning. Nature, 2015.  
[2] Schaul, Tom, et al. Prioritized experience replay. arXiv preprint arXiv:1511.05952, 2015.  
[3] http://blog.varunajayasiri.com/ml/dqn.html, last visited 2020-04-18  
[4] https://github.com/openai/baselines/blob/master/baselines/common/segment_tree.py, last visited 2020-04-18
[5] https://www.fcodelabs.com/2019/03/18/Sum-Tree-Introduction/, last visited 2020-04-18
[6] Osband, Ian, et al. "Behaviour Suite for Reinforcement Learning." arXiv preprint arXiv:1908.03568, 2019.  


The figures here were produced by the analysis Jupyter Notebook from [the BSuite code repository](https://github.com/deepmind/bsuite).